In [66]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta
import os
import pytz

api_key = ''
api_secret = ''
client = Client(api_key, api_secret)

symbol = 'DOGEUSDT'
interval = Client.KLINE_INTERVAL_1HOUR

# 使用 UTC 时间
start_time = datetime(2024, 12, 29, tzinfo=pytz.UTC)
start_str = start_time.strftime('%Y-%m-%d %H:%M:%S')

# 获取 960 条小时数据（40天）
klines = client.get_historical_klines(
    symbol=symbol,
    interval=interval,
    start_str=start_str,
    limit=960  # 不要传 end_str
)

print(f"klines 返回了 {len(klines)} 条数据")

df = pd.DataFrame(klines, columns=[
    "timestamp", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "num_trades",
    "taker_buy_base", "taker_buy_quote", "ignore"
])

if df.empty:
    print("❌ 没有获取到数据，请检查时间或交易对")
else:
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
    df = df[["timestamp", "open", "high", "low", "close", "volume"]]
    df.set_index("timestamp", inplace=True)

    last_date = df.index[-1].strftime("%Y%m%d")
    save_dir = "Dogecoin_Price"
    os.makedirs(save_dir, exist_ok=True)
    file_name = f"Dogecoin_hourly_ohlcv_until_{last_date}.csv"
    file_path = os.path.join(save_dir, file_name)

    df.to_csv(file_path)
    print(f"✅ 数据保存至: {file_path}")
    print(df.tail())


klines 返回了 960 条数据
✅ 数据保存至: Dogecoin_Price/Dogecoin_hourly_ohlcv_until_20250206.csv
                           open        high         low       close  \
timestamp                                                             
2025-02-06 19:00:00  0.24508000  0.24861000  0.24274000  0.24388000   
2025-02-06 20:00:00  0.24388000  0.25066000  0.24334000  0.25045000   
2025-02-06 21:00:00  0.25045000  0.25051000  0.24604000  0.24878000   
2025-02-06 22:00:00  0.24879000  0.25083000  0.24588000  0.25029000   
2025-02-06 23:00:00  0.25028000  0.25057000  0.24614000  0.24789000   

                                volume  
timestamp                               
2025-02-06 19:00:00  60047072.00000000  
2025-02-06 20:00:00  39430469.00000000  
2025-02-06 21:00:00  39187741.00000000  
2025-02-06 22:00:00  43246386.00000000  
2025-02-06 23:00:00  44042637.00000000  


In [67]:
import pandas as pd
import glob
import os

# 设置目标目录
folder_path = "/home/jesse/Projects/myprojs/MT_1/01_Data_Collection/Dogecoin_Price"

# 获取所有 CSV 文件路径
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 初始化空列表用于存储数据帧
dataframes = []

# 读取每个 CSV 文件并添加到列表中
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# 合并所有数据帧
combined_df = pd.concat(dataframes, ignore_index=True)

# 确保 'timestamp' 列为 datetime 类型
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])

# 筛选出截至 2024 年 12 月 31 日的数据
combined_df = combined_df[combined_df['timestamp'] <= '2024-12-31']

# 去除重复的时间戳记录，保留首次出现的记录
combined_df = combined_df.drop_duplicates(subset='timestamp', keep='first')

# 重置索引
combined_df.reset_index(drop=True, inplace=True)

# 保存合并后的数据到新的 CSV 文件
output_path = os.path.join(folder_path, "Dogecoin_combined_2019-07-05_to_2024-12-31.csv")
combined_df.to_csv(output_path, index=False)

print(f"✅ 合并完成，文件已保存至: {output_path}")


✅ 合并完成，文件已保存至: /home/jesse/Projects/myprojs/MT_1/01_Data_Collection/Dogecoin_Price/Dogecoin_combined_2019-07-05_to_2024-12-31.csv
